In [2]:
import sc2reader
from sc2reader.engine.plugins import APMTracker, ContextLoader, SelectionTracker
from sc2reader import events
import pandas as pd
import glob


replays = []
paths = [path for path in glob.glob('./_data/**/*.SC2Replay', recursive=True)]

count = 10

for i, path in enumerate(paths[:count]):
    print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)
    
    replay = sc2reader.load_replay(path, engine=sc2reader.engine.GameEngine(plugins=[ContextLoader(), APMTracker(), SelectionTracker()]))
    replays.append(replay)

100.00% complete

In [ ]:
for event in replays[0].messages:
    print('%s\tmessage="%s"' % (event, event.text))

replays[0].players[0].name
print(replays[0].players[0], replays[0].players[0].result)

In [4]:
data = []

attrs_PlayerStatsEvent = ['frame','second','workers_active_count','minerals_collection_rate','vespene_collection_rate']
# attrs_UnitPositionsEvent = ['ability_type']

for i, replay in enumerate(replays[:2]):
	# print('\r%6.2f%% complete' % ((i+1)/count*100), end='', flush=True)

	super_set = set()

	d = {}
	
	set_start = False

	for event in replay.events:

		if event.second % 30 == 0:

			# every 10 seconds
			if isinstance(event, events.PlayerStatsEvent):

				is_player_1 = replay.players[1].pid == event.pid
				name = replay.players[is_player_1].name
				race = replay.players[is_player_1].pick_race[0]
				win = replay.players[is_player_1].result == 'Win'

				lower_bound = 0 if event.second == 0 else event.second-30
				ap30s = sum(list(replay.players[is_player_1].aps.values())[lower_bound:event.second])

				d['match_id'] = i
				d['ap30s'] = ap30s
				d['race'] = race
				d['name'] = name
				d['win'] = win

				for attr in attrs_PlayerStatsEvent:
					d[attr] = eval('event.' + attr)
				
				set_start = True

			# every 15 seconds
			if isinstance(event, events.UnitPositionsEvent):
				if set_start:
					d['foobar'] = 'foobar'
					data.append(d)
					d = {}

					s = set([str(a).split(' ')[0] for a in event.units.keys()])
					super_set = super_set | s

					set_start = False

				# for attr in attrs_UnitPositionsEvent:
				# 	d[attr] = eval('event.' + attr)
				# data.append(d)

	print(super_set)
pd.DataFrame(data)

{'Colossus', 'Probe', 'SCV', 'Sentry', 'Archon', 'Disruptor', 'Hellion', 'Starport', 'Marine', 'Marauder', 'OrbitalCommand', 'Viking', 'WarpPrismPhasing', 'Stalker', 'Zealot', 'SiegeTankSieged'}
{'WidowMine', 'SCV', 'Probe', 'Colossus', 'Sentry', 'ObserverSiegeMode', 'Ghost', 'Archon', 'Raven', 'Disruptor', 'DarkTemplar', 'Marine', 'OrbitalCommand', 'Marauder', 'LiberatorAG', 'Viking', 'WidowMineBurrowed', 'OrbitalCommandFlying', 'Stalker', 'Medivac', 'SiegeTank', 'WarpPrism', 'Zealot', 'Phoenix', 'SiegeTankSieged'}


,match_id,ap30s,race,name,win,frame,second,workers_active_count,minerals_collection_rate,vespene_collection_rate,foobar
0,0,156.8,P,Probe,True,1920,120,19,895,67,foobar
1,0,124.6,P,Probe,True,2880,180,22,895,201,foobar
2,0,165.2,P,Probe,True,4320,270,25,979,335,foobar
3,0,226.8,P,Probe,True,4800,300,24,895,268,foobar
4,0,215.6,P,Probe,True,5280,330,24,979,179,foobar
5,0,162.4,P,Probe,True,5760,360,30,1119,313,foobar
6,0,224.0,P,Probe,True,8160,510,47,1763,649,foobar
7,0,359.8,P,Probe,True,9120,570,48,1931,649,foobar
8,0,162.4,P,Probe,True,9600,600,51,1903,671,foobar
9,0,212.8,P,Probe,True,10080,630,55,1875,649,foobar


In [ ]:
import pickle

with open('pieces.pickle', 'rb') as f:
    pieces = pickle.load(f)

pieces.keys()